# Save and load Keras models
# 保存与加载Keras模型

## Introduction
## 介绍
A Keras model consists of multiple components:

1. The architecture, or configuration, which specifies what layers the model contain, and how they're connected.
2. A set of weights values (the "state of the model").
3. An optimizer (defined by compiling the model).
4. A set of losses and metrics (defined by compiling the model or calling add_loss() or add_metric()).

一个Keras模型由多个部分组成：
1. 模型结构/设置。这个部分决定了模型包含了哪些层，以及这些层在模型内部的连接方式
2. 一系列的权重参数（即“模型的状态”）
3. 一个优化器（在汇编的时候加入）
4. 一系列的损失和指标信息（在汇编的时候加入模型，也可以通过调用add_loss()和add_metric()加入）


The Keras API makes it possible to save all of these pieces to disk at once, or to only selectively save some of them:

1. Saving everything into a single archive in the TensorFlow SavedModel format (or in the older Keras H5 format). This is the standard practice.
2. Saving the architecture / configuration only, typically as a JSON file.
3. Saving the weights values only. This is generally used when training the model.

Keras API可以让你一次性地将所有部分保存到本地，或者选择性地保存其中的某些部分：

1. 常规操作：通过TensorFlow的模型保存格式（或者旧版的Keras H5格式）来将所有东西保存进一个单独的文件
2. 只保存结构/设置部分，通常保存为JSON文件
3. 只保存权重部分。这通常发生在训练模型的时候

Let's take a look at each of these options. When would you use one or the other, and how do they work?

让我们依次介绍每一个选项：你应该何时使用这种方式或那种方式，以及各个方式是怎样运作的？

## How to save and load a model
## 如何保存和加载一个模型

If you only have 10 seconds to read this guide, here's what you need to know.

如果你时间紧迫，这部分就是你需要了解的内容

In [ ]:
########################
## Saving a Keras model: # 保存模型示例
########################

model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save('path/to/location')



########################
#Loading the model back: #加载模型示例
########################

from tensorflow import keras
model = keras.models.load_model('path/to/location')

# Note 学习笔记

if you have a model called "MODEL" ans you want to save it. use **MODEL.save('save path')** method

if you want to load a model to MODEL, use **keras.models.load_model('save path')** function

如果你有了一个叫“MODEL”的模型并且打算保存它，使用**MODEL.save('save path')**方法

如果你要加载一个模型到“MODEL”，使用**keras.models.load_model('save path')**函数

Now, let's look at the details.

现在，让我们深入了解其中的细节

## Setup
## 环境设置

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Whole-model saving & loading
## 模型整体的保存与加载
You can save an entire model to a single artifact. It will include:

1. The model's architecture/config
2. The model's weight values (which were learned during training)
3. The model's compilation information (if compile() was called)
4. The optimizer and its state, if any (this enables you to restart training where you left)

**APIs**
1. model.save() or tf.keras.models.save_model()
2. tf.keras.models.load_model()

如果你要保存整个模型，你需要保存的是：
1. 模型的结构和设置
2. 模型的权重（这是训练过程中，模型从资料中‘学到’的部分）
3. 模型的汇编信息（如果调用过compole（）的话）
4. 优化器及其状态，如果有的话（这可以让你从上次终端的地方重新开始训练）

**API实现**
1. 保存模型：model.save() or tf.keras.models.save_model()
2. 加载模型：tf.keras.models.load_model()

There are two formats you can use to save an entire model to disk: the TensorFlow SavedModel format, and the older Keras H5 format. The recommended format is SavedModel. It is the default when you use model.save().

You can switch to the H5 format by:

1. Passing save_format='h5' to save().
2. Passing a filename that ends in .h5 or .keras to save().

有两种格式可以用于保存整个模型到本地硬盘上：“SavedModel”格式和旧版”Keras H5“格式。 推荐使用“SavedModel”格式。 当你使用model.save()时，默认使用的就是“SavedModel”格式

你也可以转换为H5格式：
1. 在save()加入“save_format='h5'”。即model.save(save_format='h5')
2. 在save()将保存的文件名后缀设置为“.h5”或“.kears”.即model.save('../user/model/model.h5')

### SavedModel format
### “SavedModel”格式

SavedModel is the more comprehensive save format that saves the model architecture, weights, and the traced Tensorflow subgraphs of the call functions. This enables Keras to restore both built-in layers as well as custom objects.

“SavedModel”格式是一种更为强大的保存格式。它可以保存模型的结构，权重，以及追踪调用函数的子结构。这使得Keras可以恢复内置层和自定义对象

In [25]:
# Example

def get_model():
    # Create a simple model.
    # 构建一个模型生成函数
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
# 训练这个模型
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
# 调用`save('my_model')`，创建一个'my_model'文件夹
model.save("my_model")

# It can be used to reconstruct the model identically.
# 这可以用来创建相同的模型
reconstructed_model = keras.models.load_model("my_model")

# Let's check:
# 检查两个模型是否一致
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
# 这个重建的模型已经包含了优化器和状态信息，所以训练可以继续
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 1ms/step - loss: 1.7330
INFO:tensorflow:Assets written to: my_model\assets
4/4 [==============================] - 1s 2ms/step - loss: 1.5573


**What the SavedModel contains**

Calling model.save('my_model') creates a folder named my_model, containing the following:

**“SavedModel”里都包含了什么**

调用model.save('my_model')名字是'my_model'的文件夹，它里面包括这些东西：

In [5]:
ls my_model

 Volume in drive C has no label.
 Volume Serial Number is 7411-1CC8

 Directory of C:\Users\carlo\Dropbox\NU\Tensorflow-Keras-tutorial\Keras\my_model

2021/04/14  02:51    <DIR>          .
2021/04/14  02:51    <DIR>          ..
2021/04/14  02:51    <DIR>          assets
2021/04/14  02:51            45,971 saved_model.pb
2021/04/14  02:51    <DIR>          variables
               1 File(s)         45,971 bytes
               4 Dir(s)  97,736,974,336 bytes free


The model architecture, and training configuration (including the optimizer, losses, and metrics) are stored in saved_model.pb. The weights are saved in the variables/ directory.

For detailed information on the SavedModel format, see [the SavedModel guide (The SavedModel format on disk).](https://www.tensorflow.org/guide/saved_model#the_savedmodel_format_on_disk)

模型结构，训练设置（包含优化器，损失和指标）信息都保存在saved_model.pb。 权重保存在“variables”字典里

要了解关于‘SavedModel’格式的更多信息，请参考[这里](https://www.tensorflow.org/guide/saved_model#the_savedmodel_format_on_disk)

**How SavedModel handles custom objects**
**SavedModel如何处理自定义对象**

When saving the model and its layers, the SavedModel format stores the class name, call function, losses, and weights (and the config, if implemented). The call function defines the computation graph of the model/layer.

In the absence of the model/layer config, the call function is used to create a model that exists like the original model which can be trained, evaluated, and used for inference.

Nevertheless, it is always a good practice to define the get_config and from_config methods when writing a custom model or layer class. This allows you to easily update the computation later if needed. See the section about Custom objects for more information.

当保存模型和它的层时，SavedModel格式会储存类的名称，call函数，损失函数，和权重（以及设置，如果已经实现了）。call函数定义了模型/层的计算次序

如果没有模型或者层设置信息，call函数会被用于创建一个和原模型相似的新模型，这个新模型同样可以被训练，评估以及用于预测

然后，在构建自定义对象时，定义好get_config以及from_config方法是在编写模型和层类时的好习惯。如果需要的话，这将允许你轻松的更新之后的计算过程。要了解更多请参见之后的“自定义对象”一节

In [6]:
# Example

class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

    def get_config(self):
        return {"hidden_units": self.hidden_units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)


model = CustomModel([16, 16, 10])
# Build the model by calling it
# 通过调用模型类来创建模型）
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("my_model") # 保存了这个模型

# Option 1: Load with the custom_object argument.
# 选择2： 加载模型，同时也加载了自定义对象的参数
loaded_1 = keras.models.load_model(
    "my_model", custom_objects={"CustomModel": CustomModel}
)

# Option 2: Load without the CustomModel class.
# 选项2： 值加载模型，不加载自定义对象的参数

# Delete the custom-defined model class to ensure that the loader does not have
# access to it.
# 删除自定义层类来确保加载的时候不会访问它
del CustomModel

loaded_2 = keras.models.load_model("my_model")
np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print("Original model:", model)
print("Model Loaded with custom objects:", loaded_1)
print("Model loaded without the custom object class:", loaded_2)

INFO:tensorflow:Assets written to: my_model\assets
Original model: <__main__.CustomModel object at 0x0000024AF234F100>
Model Loaded with custom objects: <__main__.CustomModel object at 0x0000024AF2696790>
Model loaded without the custom object class: <tensorflow.python.keras.saving.saved_model.load.CustomModel object at 0x0000024AF29F76A0>


The first loaded model is loaded using the config and CustomModel class. The second model is loaded by dynamically creating the model class that acts like the original model.

第一个加载的模型里包含了自定义模型类的设置信息。第二个加载的模型里动态的创建了一个模型类，其表现和原模型类似

**Configuring the SavedModel**

New in TensoFlow 2.4 The argument save_traces has been added to model.save, which allows you to toggle SavedModel function tracing. Functions are saved to allow the Keras to re-load custom objects without the original class definitons, so when save_traces=False, all custom objects must have defined get_config/from_config methods. When loading, the custom objects must be passed to the custom_objects argument. save_traces=False reduces the disk space used by the SavedModel and saving time.

**SavedModel设置**

TensoFlow 2.4的一个新功能是给model.save（）添加了新参数“save_traces”。这个允许你更改SavedModel函数对保存对象的检查方式。 模型被保存是为了让Keras在缺少原始类定义的条件下重新加载自定义对象。所以，如果“save_traces=False”，那么所有的自定义对象都必须有定义好的“get_config/from_config”方法。在加载模型的时候，这些自定义对象都会被传递给“custom_objects”参数。“save_traces=False”可以减少在通过SavedModel格式保存时的保存空间和耗时

### Keras H5 format
Keras also supports saving a single HDF5 file containing the model's architecture, weights values, and compile() information. It is a light-weight alternative to SavedModel.

### Keras H5格式
Keras同样支持将模型的结构，权重，汇编信息保存为单独的HDF5文件，它是SavedModel的轻量级替代品。

In [26]:
# Example

model = get_model()

# Train the model.
#训练模型
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
# 保存为H5文件
model.save("my_h5_model.h5")

# It can be used to reconstruct the model identically.
# 重建一个相同的模型
reconstructed_model = keras.models.load_model("my_h5_model.h5")

# Let's check:
# 检查
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
# 这个重建的模型已经包含了优化器和状态信息，所以它可以训练可以继续
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 2ms/step - loss: 1.0419


**Limitations**

Compared to the SavedModel format, there are two things that don't get included in the H5 file:

1. External losses & metrics added via model.add_loss() & model.add_metric() are not saved (unlike SavedModel). If you have such losses & metrics on your model and you want to resume training, you need to add these losses back yourself after loading the model. Note that this does not apply to losses/metrics created inside layers via self.add_loss() & self.add_metric(). As long as the layer gets loaded, these losses & metrics are kept, since they are part of the call method of the layer.
2. The computation graph of custom objects such as custom layers is not included in the saved file. At loading time, Keras will need access to the Python classes/functions of these objects in order to reconstruct the model. See Custom objects.

**局限**
和SavedModel相比，H5有两件事无法做到：
1. H5无法保存通过model.add_loss() 和 model.add_metric()传递的损失函数和指标信息。如果你的模型含有这些信息并且你打算在模型回复后继续训练，你必须手动添加这些信息。 注意，由模型内部层创建的损失函数和指标信息是可以保存下来的。这是因为这些信息是call方法的一部分，一旦层被加载，这些信息也就保存了下来
2. 自定义对象结构如自定义层是不包含在保存文件里的。当加载模型时，Keras会访问这些对象的pyhton类或者函数来重建模型。请参阅“自定义对象”一节

## Saving the architecture

The model's configuration (or architecture) specifies what layers the model contains, and how these layers are connected*. If you have the configuration of a model, then the model can be created with a freshly initialized state for the weights and no compilation information.

*Note this only applies to models defined using the functional or Sequential apis not subclassed models.

## 保存结构

模型的设置（或者说结构）决定了模型含有哪些层，以及这些层是如何相互连接的。如果你有模型的设置信息，哪些这个模型就可以在初始化权重状态下创建模型（即丢失训练信息）而不需要汇编信息

**注意**这只对顺序模型和函数式模型有效，对子类模型无效

### Configuration of a Sequential model or Functional API model
### 顺序模型或函数式API模型的设置

These types of models are explicit graphs of layers: their configuration is always available in a structured form.

这两类模型有着明确的层结构： 他们的设置永远是结构化的

**APIs**
1. get_config() and from_config()
2. tf.keras.models.model_to_json() and tf.keras.models.model_from_json()

**get_config() and from_config()**

Calling config = model.get_config() will return a Python dict containing the configuration of the model. The same model can then be reconstructed via Sequential.from_config(config) (for a Sequential model) or Model.from_config(config) (for a Functional API model).

The same workflow also works for any serializable layer.

**APIs**
1. get_config()和from_config()
2. tf.keras.models.model_to_json() 和 tf.keras.models.model_from_json()

**et_config()和from_config()**

设置为config = model.get_config()时，会返回包含模型设置信息的python字典；模型可以通过“Sequential.from_config(config)”（顺序模型适用）或者“Model.from_config(config)”（函数式API适用）

这个流程对任意次序化层也同样适用

In [ ]:
###############
#Layer example:
###############

layer = keras.layers.Dense(3, activation="relu")
layer_config = layer.get_config()
new_layer = keras.layers.Dense.from_config(layer_config)



##########################
#Sequential model example:
##########################

model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
config = model.get_config()
new_model = keras.Sequential.from_config(config)



##########################
#Functional model example:
##########################

inputs = keras.Input((32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
config = model.get_config()
new_model = keras.Model.from_config(config)

# Note
For build-in layers, sequential model, and functional model, these objects can get configuration information by get_config() and recreated it by from_config(). Like this:
1. config_inforamtion = layers.get_config()/Suqential_Model.get_config()/Functional_Model.get_config()
2. new_layer_or Model = keras.layers.Dense.from_config(config_inforamtion)/eras.Sequential.from_config(config_inforamtion)/ keras.Model.from_config(config_inforamtion)

对于内置的层，顺序模型和函数式API，可以通过get_config()和from_config()实现提取设置信息并重建模型或层。就像这样
1. 模型信息 = layers.get_config()/Suqential_Model.get_config()/Functional_Model.get_config()
2. 新的模型或层 =  keras.layers.Dense.from_config(config_inforamtion)/eras.Sequential.from_config(config_inforamtion)/ keras.Model.from_config(config_inforamtion)

**to_json() and tf.keras.models.model_from_json()**

This is similar to get_config / from_config, except it turns the model into a JSON string, which can then be loaded without the original model class. It is also specific to models, it isn't meant for layers.

**to_json()和tf.keras.models.model_from_json()方法**

这两个方法和get_config / from_config类似，区别在于他们会将模型信息转化为JSON格式，这样及时没有源模型类也能加载模型。不过这只适用于模型，不适用于层

In [7]:
# Example

model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
json_config = model.to_json()
new_model = keras.models.model_from_json(json_config)

### Custom objects
***Models and layers***

The architecture of subclassed models and layers are defined in the methods \__init__ and call. They are considered Python bytecode, which cannot be serialized into a JSON-compatible config -- you could try serializing the bytecode (e.g. via pickle), but it's completely unsafe and means your model cannot be loaded on a different system.

In order to save/load a model with custom-defined layers, or a subclassed model, you should overwrite the get_config and optionally from_config methods. Additionally, you should use register the custom object so that Keras is aware of it.

### 自定义对象

**层与模型**

子类模型和层的结构是通过__init__和 call定义的。这些对象会被识别为python字节码，所以无法被序列化并保存为JSON兼容配置。你可以尝试去序列化这些字节码（比如通过pickle），但是这不安全，同时也意味着你的模型无法在其他系统下加载

为了保存含有自定义层的模型或者自定义模型， 你可以重写getget_config和optionally from_config（有需要的话）两个方法，并用新方法覆盖旧的。此外，你可以注册这些自定义对象，这样Keras就能识别它们了

**Custom functions**

Custom-defined functions (e.g. activation loss or initialization) do not need a get_config method. The function name is sufficient for loading as long as it is registered as a custom object.

**自定义函数**

自定义函数（如激活函数和初始化函数）并不需要get_config。只要这些函数已经注册，那么函数名已经足够用于恢复

**Loading the TensorFlow graph only**

It's possible to load the TensorFlow graph generated by the Keras. If you do so, you won't need to provide any custom_objects. You can do so like this:

**只加载TensorFlow层次结构**

通过Keras只加载TensorFlow层次结构是可行的。如果你这么做，你不需要提供任何自定义对象。 你可以像这样实现这一功能：

In [8]:
model.save("my_model")
tensorflow_graph = tf.saved_model.load("my_model")
x = np.random.uniform(size=(4, 32)).astype(np.float32)
predicted = tensorflow_graph(x).numpy()

INFO:tensorflow:Assets written to: my_model\assets


# Note 学习笔记
when we load the whole model from "my model", we use 'keras.models.load_model()' function; Here, we just load the graph and we use "tf.saved_model.load()". So, loading model or part of model depends on which function we choose.

当我们加载整个模型时，我们使用的是'keras.models.load_model()'函数；而在这里我们只加载层次结构，我们使用的是"tf.saved_model.load()"。所以加载全部模型或部分模型成分取决于我们所使用的函数
***

Note that this method has several drawbacks:

1. For traceability reasons, you should always have access to the custom objects that were used. You wouldn't want to put in production a model that you cannot re-create.
2. The object returned by tf.saved_model.load isn't a Keras model. So it's not as easy to use. For example, you won't have access to .predict() or .fit()

注意这一方法有以下缺点：
1. 从可追踪性的角度出发，你应该可以访问任何被使用的自定义对象。你也不会想要创建一个无法再次被创建的模型
2. tf.saved_model.load（）所返回的对象并不是一个Keras模型，所以这并不好用。比如说，你无法在这个模型上调用.predict()或者.fit()

Even if its use is discouraged, it can help you if you're in a tight spot, for example, if you lost the code of your custom objects or have issues loading the model with tf.keras.models.load_model().

You can find out more in the [page about](https://www.tensorflow.org/api_docs/python/tf/saved_model/load) tf.saved_model.load

虽然不鼓励使用这种方法，不过它还是可以帮你应急的。比如你丢失了你自定义对象的代码，或者当你在使用tf.keras.models.load_model()加载模型时遇到的问题

你可以在[这里](https://www.tensorflow.org/api_docs/python/tf/saved_model/load)找到更多关于tf.saved_model.load的信息

**Defining the config methods**

Specifications:

1. get_config should return a JSON-serializable dictionary in order to be compatible with the Keras architecture- and model-saving APIs.
2. from_config(config) (classmethod) should return a new layer or model object that is created from the config. The default implementation returns cls(**config).

**定义config方法**

具体来说
1. get_config会返回一个JSON次序化的字典来兼容Keras结构和模型保存API
2. from_config(config)（类方法）则会返回一个新的，从config中创建的层或者模型对象。默认实现会返回cls(**config)

In [ ]:
# exmaple

class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")

    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs

    def get_config(self):
        return {"a": self.var.numpy()}

    # There's actually no need to define `from_config` here, since returning
    # `cls(**config)` is the default behavior.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


layer = CustomLayer(5)
layer.var.assign(2)

serialized_layer = keras.layers.serialize(layer)
new_layer = keras.layers.deserialize(
    serialized_layer, custom_objects={"CustomLayer": CustomLayer}

**Registering the custom object**

Keras keeps a note of which class generated the config. From the example above, tf.keras.layers.serialize generates a serialized form of the custom layer:

**注册自定义对象**

Keras有一个“记事本”，这个记事本记录了哪些类可以生成配置信息。在上面的示例中，tf.keras.layers.serialize就生成了一个自定义对象的序列化格式：

In [ ]:
{'class_name': 'CustomLayer', 'config': {'a': 2} }

Keras keeps a master list of all built-in layer, model, optimizer, and metric classes, which is used to find the correct class to call from_config. If the class can't be found, then an error is raised (Value Error: Unknown layer). There are a few ways to register custom classes to this list:

1. Setting custom_objects argument in the loading function. (see the example in section above "Defining the config methods")
2. tf.keras.utils.custom_object_scope or tf.keras.utils.CustomObjectScope
3. tf.keras.utils.register_keras_serializable

Keras保存了所有内置层，模型，优化器和指标类的主要列表。这个列表可以用来为from_config查找正确的类。如果没有找到对应的类，则会返回一个错误信息。这里有几个将自定义模型注册登记在这个列表里的办法：
1. 在加载函数的时候设置自定义对象的参数（参见**定义config方法**一节的示例）
2. tf.keras.utils.custom_object_scope或者tf.keras.utils.CustomObjectScope
3. tf.keras.utils.register_keras_serializable

**Custom layer and function example**

**自定义层和模型示例**

In [27]:
# Example

class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config


def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# Make a model with the CustomLayer and custom_activation
# 用自定义层和自定义激活函数来构建一个模型
inputs = keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(x)
model = keras.Model(inputs, outputs)

# Retrieve the config
# 读取配置信息
config = model.get_config()

# At loading time, register the custom objects with a `custom_object_scope`:
# 在读取的时候，将自定义对象注册为一个“custom_object_scope”
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation} # 生成自定义对象列表
with keras.utils.custom_object_scope(custom_objects): # 通过keras.utils.custom_object_scope进行注册
    new_model = keras.Model.from_config(config) # 根据配置信息（config）重建模型

# Note 学习笔记

In my opinion, building model looks like building house with two type of materials (build-in objects and custom objects).

when we recreate a model based on config inforamtion, it means we ask Keras re-build a house based on our material list. However,the keras only search material in the build-in object list. So, if our model is built with costum objects, the keras cannot find it to rebuild. Therefore, we can add the custom objects into the build-in list, or warp the custom object with a build-in objects to help Keras rebuild a now house successfully.

在我看来，建模就好像用两种材料（内置对象，自定义对象）建房子

当我根据配置信息重建模型时，就相当于我要求Keras根据我提供的材料清单来重建一栋房子。然后，Keras只会从内置对象列表中搜索所需要的材料。所以，如果我的模型中含有自定义对象，Keras就无法找到我所要求的材料来重建房子。 因此，我们可以通过将自定义对象加入内置对象列表（方式一）或者用内置对象来包装自定义对象的方式（方式二）来帮助Keras成功的重建房子。

### In-memory model cloning
### 内存模型克隆

You can also do in-memory cloning of a model via tf.keras.models.clone_model(). This is equivalent to getting the config then recreating the model from its config (so it does not preserve compilation information or layer weights values).

你可以通过tf.keras.models.clone_model()实现在内存中克隆一个模型。这相当于得到配置信息并马上基于此信息重建模型（所以不含任何汇编信息或者层的权重信息）

In [11]:
# Example
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

## Saving & loading only the model's weights values
You can choose to only save & load a model's weights. This can be useful if:

1. You only need the model for inference: in this case you won't need to restart training, so you don't need the compilation information or optimizer state.
2. You are doing transfer learning: in this case you will be training a new model reusing the state of a prior model, so you don't need the compilation information of the prior model.

## 仅保存或加载模型的权重信息

你可以选择仅保存或在家模型的权重信息，这在以下情况下非常有用：
1. 你只需要模型用于预测：这种情况下，你不需要重新训练模型，所以你也不需要汇编信息和优化器状态
2. 你在做迁移学习：这种情况下，你只需要前一个模型的状态信息用于训练下一个新模型，所以你不需要前一个模型的汇编信息


### APIs for in-memory weight transfer
### 用于内存权重迁移的APIs

Weights can be copied between different objects by using get_weights and set_weights:

1. tf.keras.layers.Layer.get_weights(): Returns a list of numpy arrays.
2. tf.keras.layers.Layer.set_weights(): Sets the model weights to the values in the weights argument.

using get_weights和set_weights可以复制不同对象间的权重信息
1. tf.keras.layers.Layer.get_weights(): 返回Numpy数组列表
2. tf.keras.layers.Layer.set_weights(): 将模型权重设置为权重参数的值

In [ ]:
###################################################################
# Example: Transfering weights from one layer to another, in memory
###################################################################
def create_layer():
    layer = keras.layers.Dense(64, activation="relu", name="dense_2")
    layer.build((None, 784))
    return layer


layer_1 = create_layer()
layer_2 = create_layer()

# Copy weights from layer 1 to layer 2
layer_2.set_weights(layer_1.get_weights())



########################################################################################################
# Example: Transfering weights from one model to another model with a compatible architecture, in memory
########################################################################################################
# Create a simple functional model
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# Define a subclassed model with the same architecture
class SubclassedModel(keras.Model):
    def __init__(self, output_dim, name=None):
        super(SubclassedModel, self).__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation="relu", name="dense_1")
        self.dense_2 = keras.layers.Dense(64, activation="relu", name="dense_2")
        self.dense_3 = keras.layers.Dense(output_dim, name="predictions")

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}


subclassed_model = SubclassedModel(10)
# Call the subclassed model once to create the weights.
subclassed_model(tf.ones((1, 784)))

# Copy weights from functional_model to subclassed_model.
subclassed_model.set_weights(functional_model.get_weights())

assert len(functional_model.weights) == len(subclassed_model.weights) # 如果两者的权重数量不同，报错
for a, b in zip(functional_model.weights, subclassed_model.weights): # 如果两者的权重数值不同，报错
    np.testing.assert_allclose(a.numpy(), b.numpy())


**The case of stateless layers**

Because stateless layers do not change the order or number of weights, models can have compatible architectures even if there are extra/missing stateless layers.

**无状态层的情况**

因为无状态层不会改变权重的顺序和数值，模型可以具有兼容结构，无论其有没有无状态层

In [12]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)

# Add a dropout layer, which does not contain any weights.
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model_with_dropout = keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

functional_model_with_dropout.set_weights(functional_model.get_weights())

### APIs for saving weights to disk & loading them back
### 用于将权重保存到本地和从本次加载权重的APIs

Weights can be saved to disk by calling model.save_weights in the following formats:

1. TensorFlow Checkpoint
2. HDF5

The default format for model.save_weights is TensorFlow checkpoint. There are two ways to specify the save format:

1. save_format argument: Set the value to save_format="tf" or save_format="h5".
2. path argument: If the path ends with .h5 or .hdf5, then the HDF5 format is used. Other suffixes will result in a TensorFlow checkpoint unless save_format is set.

权重可以通过调用model.save_weights来保存到本地，保存的格式有：
1. TensorFlow 检查点
2. HDF5

model.save_weights的默认保存格式是TensorFlow 检查点。有两种方式可以指定特定的保存格式：
1. 通过save_format参数指定：设定参数的值为save_format="tf"或者save_format="h5"
2. 通过路径参数指定：如果路径后缀为.h5 或者.hdf5，则保存为HDF5格式，其他路径则保存为TensorFlow 检查点格式，除非同时还设置了save_format参数

# Note

like saving and laoding model, there are two ways  to save and load weights:
1. TensorFlow formart
2. H5 formart

Also, we can set the preferred saving format by two ways:
1. set argument directly
2. set path argument

和模型的保存与加载相同，有两种方式可以保存和加载权重：
1. TensorFlow格式
2. H5格式

同样的，我们有两种方式来设置偏好的保存格式：
1. 直接设置对应的参数
2. 设置保存路径参数

***

There is also an option of retrieving weights as in-memory numpy arrays. Each API has its pros and cons which are detailed below.
同样可以选择将权重读取为Numpy数组，可选择的API以及优缺点如下：

**TF Checkpoint format**
**TF 检查点格式**

In [13]:
# Example

# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("ckpt")
load_status = sequential_model.load_weights("ckpt")

# `assert_consumed` can be used as validation that all variable values have been
# restored from the checkpoint. See `tf.train.Checkpoint.restore` for other
# methods in the Status object.
load_status.assert_consumed()

**Format details**

The TensorFlow Checkpoint format saves and restores the weights using object attribute names. For instance, consider the tf.keras.layers.Dense layer. The layer contains two weights: dense.kernel and dense.bias. When the layer is saved to the tf format, the resulting checkpoint contains the keys "kernel" and "bias" and their corresponding weight values. For more information see ["Loading mechanics" in the TF Checkpoint guide.](https://www.tensorflow.org/guide/checkpoint#loading_mechanics)

Note that attribute/graph edge is named after the name used in parent object, not the name of the variable. Consider the CustomLayer in the example below. The variable CustomLayer.var is saved with "var" as part of key, not "var_a".

**格式细节**
TensorFlow检查点格式会根据属性名称对权重进行保存会恢复。例如，有一个tf.keras.layers.Dense层（稠密层）。这个层有两个权重：dense.kernel和dense.bias. 当这个层以检查点格式保存权重时，所产生的检查点会包含“kernel”和"bias"两个名称以及对应的权重。更多信息请参见[这里](https://www.tensorflow.org/guide/checkpoint#loading_mechanics)

注意，数据/层级是根据其父对象的名字命名的，而不是这个变量本身。请参考以下示例。变量CustomLayer.var用“var”作为键的一部分保存，而不是“var_a”

In [29]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")


layer = CustomLayer(5)
layer_ckpt = tf.train.Checkpoint(layer=layer).save("custom_layer")

ckpt_reader = tf.train.load_checkpoint(layer_ckpt)

ckpt_reader.get_variable_to_dtype_map()

{'_CHECKPOINTABLE_OBJECT_GRAPH': tf.string,
 'layer/var/.ATTRIBUTES/VARIABLE_VALUE': tf.int32,
 'save_counter/.ATTRIBUTES/VARIABLE_VALUE': tf.int64}

# Note 学习笔记
The official explanation is so comfused. My idea is that although we give the CustomLayer.var a name by tf.Variable(a, name="var_a"), but this action will not impact the expression of its key. No matter what name the function given, the key always set "var" as a part of key. If you want to change it, you should change the "self.var" part.

you can try "self.var_test" or "self.this_is_not_a_variable" in the above example and see how the key change.

官方的解释有点难懂。我的理解是，尽管我们通过 tf.Variable(a, name="var_a")对这个CustomLayer.var进行了命名，但是这个行为不会改变保存时候的键值。不论我们怎么通过 tf.Variable(a, name="var_a")传递名称，这个键值永远会用“var”来作为其一部分。 如果要变量这部分键值，你应该修改"self.var"部分。

你可以在上面的例子中试试"self.var_test"或者"self.this_is_not_a_variable"，然后看看键值会如何改变

**Transfer learning example**

Essentially, as long as two models have the same architecture, they are able to share the same checkpoint.

**迁移学习示例**

基本上，如果两个模型有着一样的结构，那么他们就可以共享同一个检查点

In [30]:
# Example

inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# Extract a portion of the functional model defined in the Setup section.
# 抽取一部分在环境设定部分定义的函数式模型

# The following lines produce a new model that excludes the final output
# layer of the functional model.
# 接来下的模型生成了一个包含函数式模型输出层的新模型
pretrained = keras.Model(
    functional_model.inputs, functional_model.layers[-1].input, name="pretrained_model"
)
# Randomly assign "trained" weights.
# 随机的分配“训练”权重
for w in pretrained.weights:
    w.assign(tf.random.normal(w.shape))
pretrained.save_weights("pretrained_ckpt")
pretrained.summary()

# Assume this is a separate program where only 'pretrained_ckpt' exists.
# Create a new functional model with a different output dimension.
# 假设这是一个单独的程序，只有'pretrained_ckpt'存在（只能通过'pretrained_ckpt'获得权重）
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(5, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="new_model")

# Load the weights from pretrained_ckpt into model.
# 从pretrained_ckpt加载权重到模型中
model.load_weights("pretrained_ckpt")

# Check that all of the pretrained weights have been loaded.
# 检查所有被加载的的权重
for a, b in zip(pretrained.weights, model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

print("\n", "-" * 50)
model.summary()

# Example 2: Sequential model
# Recreate the pretrained model, and load the saved weights.
# 重建一个未经训练的模型，并加入保存的权重信息
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
pretrained_model = keras.Model(inputs=inputs, outputs=x, name="pretrained")

# Sequential example:
# 顺序模型示例
model = keras.Sequential([pretrained_model, keras.layers.Dense(5, name="predictions")])
model.summary()

pretrained_model.load_weights("pretrained_ckpt")

# Warning! Calling `model.load_weights('pretrained_ckpt')` won't throw an error,
# but will *not* work as expected. If you inspect the weights, you'll see that
# none of the weights will have loaded. `pretrained_model.load_weights()` is the
# correct method to call.

# 注意：调用model.load_weights('pretrained_ckpt')不会导致错误，但是它也不会如你预期那样工作：没有权重被载入。
# `pretrained_model.load_weights()`才是正确的调用方法

Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________

 --------------------------------------------------
Model: "new_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64) 

It is generally recommended to stick to the same API for building models. If you switch between Sequential and Functional, or Functional and subclassed, etc., then always rebuild the pre-trained model and load the pre-trained weights to that model.

The next question is, how can weights be saved and loaded to different models if the model architectures are quite different? The solution is to use tf.train.Checkpoint to save and restore the exact layers/variables.

通常建议用相同的API来构建模型。如果你在顺序API和函数式APS之间或者函数式API和子类之间切换使用，则会重建一个未经训练的模型，并且载入未经训练的权重到这个模型中

另一个问题是，如果模型的结构差异较大，那么权重如何保存并加载到不同的模型之中呢。解决方案是使用tf.train.Checkpoint来保存和加载额外的层和变量

In [17]:
# Example

# Create a subclassed model that essentially uses functional_model's first
# and last layers.
# First, save the weights of functional_model's first and last dense layers.
first_dense = functional_model.layers[1]
last_dense = functional_model.layers[-1]
ckpt_path = tf.train.Checkpoint(
    dense=first_dense, kernel=last_dense.kernel, bias=last_dense.bias
).save("ckpt")

# Define the subclassed model.
class ContrivedModel(keras.Model):
    def __init__(self):
        super(ContrivedModel, self).__init__()
        self.first_dense = keras.layers.Dense(64)
        self.kernel = self.add_variable("kernel", shape=(64, 10))
        self.bias = self.add_variable("bias", shape=(10,))

    def call(self, inputs):
        x = self.first_dense(inputs)
        return tf.matmul(x, self.kernel) + self.bias


model = ContrivedModel()
# Call model on inputs to create the variables of the dense layer.
_ = model(tf.ones((1, 784)))

# Create a Checkpoint with the same structure as before, and load the weights.
tf.train.Checkpoint(
    dense=model.first_dense, kernel=model.kernel, bias=model.bias
).restore(ckpt_path).assert_consumed()

C:\Users\carlo\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


### HDF5 format

The HDF5 format contains weights grouped by layer names. The weights are lists ordered by concatenating the list of trainable weights to the list of non-trainable weights (same as layer.weights). Thus, a model can use a hdf5 checkpoint if it has the same layers and trainable statuses as saved in the checkpoint.

### HDF5 格式

HDF5 格式将权重按层名称打包。这些权重按照从可训练权重到不可训练权重的次序列出（层权重也是如此）。因此模型可以使用HDF5格式检查点，如果这个模型和检查点有着相同的层和“可训练”状态

In [19]:
# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("weights.h5")
sequential_model.load_weights("weights.h5")

## 这个模型的所有层都是稠密层，并且没有不可训练的权重，所以可以通过HDF5格式保存和恢复权重

Note that changing layer.trainable may result in a different layer.weights ordering when the model contains nested layers.

注意，当模型包含层嵌套结构时，改变层的可训练状态会导致一个不同的层权重排序。

In [31]:
class NestedDenseLayer(keras.layers.Layer):
    def __init__(self, units, name=None):
        super(NestedDenseLayer, self).__init__(name=name)
        self.dense_1 = keras.layers.Dense(units, name="dense_1")
        self.dense_2 = keras.layers.Dense(units, name="dense_2")

    def call(self, inputs):
        return self.dense_2(self.dense_1(inputs))

# 原始状态的嵌套模型，两个层都是可训练的。先报告layer1的权重，再报告layer2的权重
nested_model = keras.Sequential([keras.Input((784,)), NestedDenseLayer(10, "nested")])
variable_names = [v.name for v in nested_model.weights]
print("variables: {}".format(variable_names))

# 改变layer1的权重状态为不可训练
print("\nChanging trainable status of one of the nested layers...")
nested_model.get_layer("nested").dense_1.trainable = False

# 再次查看权重，权重按可训练-不可训练排序，所以layer2在前，layer1在后
variable_names_2 = [v.name for v in nested_model.weights]
print("\nvariables: {}".format(variable_names_2))
print("variable ordering changed:", variable_names != variable_names_2)

variables: ['nested/dense_1/kernel:0', 'nested/dense_1/bias:0', 'nested/dense_2/kernel:0', 'nested/dense_2/bias:0']

Changing trainable status of one of the nested layers...

variables: ['nested/dense_2/kernel:0', 'nested/dense_2/bias:0', 'nested/dense_1/kernel:0', 'nested/dense_1/bias:0']
variable ordering changed: True


**Transfer learning example**

When loading pretrained weights from HDF5, it is recommended to load the weights into the original checkpointed model, and then extract the desired weights/layers into a new model.

**迁移学习示例**

当从HDF5文件中载入未经训练的权重时，推荐的方式是先把权重载入到一个原检查点模型中，然后再提取需要的权重/层，并加入到新模型里面

In [32]:
# Example

def create_functional_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = keras.layers.Dense(10, name="predictions")(x)
    return keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# 先创建模型，然后保存权重
functional_model = create_functional_model()
functional_model.save_weights("pretrained_weights.h5")

# In a separate program:
# 又通过同样的方式创建了一个模型，并加入了之前的权重
pretrained_model = create_functional_model()
pretrained_model.load_weights("pretrained_weights.h5")

# Create a new model by extracting layers from the original model:
# 提取之前模型的层（最后一层除外），并创建第三个模型
extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name="dense_3"))
model = keras.Sequential(extracted_layers)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________
